In [14]:
#import packages
import spacy
import re
import requests
import webbrowser

# Load the SpaCy NLP model
nlp = spacy.load('en_core_web_sm')

#  dictionary responses
responses = {
    "greet": "Hello! How can I assist you today?",
    "farewell": "Goodbye! Have a great day!",
    "how are you": "I'm a bot, so I don't have feelings, but thanks for asking!",
    "name": "I'm your friendly chatbot. You can call me Bot.",
    "help": "I'm here to help! Ask me anything.",
    "learn python": "You can start with this beginner's guide to Python: https://docs.python.org/3/tutorial/",
    "tell me a joke": "Why do programmers prefer dark mode? Because light attracts bugs!",
    "tell me something interesting": "Did you know that Python was named after Monty Python, not the snake?",
    "weather": "Let me check the weather for you. Please provide the city name.",
    "calculate": "I can perform basic calculations like addition, subtraction, multiplication, and division.",
    "open python docs": "Sure! Opening the Python documentation for you.",
    "learn ai": "Check out this link to learn more about AI: https://www.coursera.org/courses?query=artificial%20intelligence"
}

# Weather API setup
API_KEY = 'cbf90cf16205e48a0393c10be774fa3e'


def get_response(user_input, context):

    doc = nlp(user_input.lower())

    # weather
    if context.get("awaiting_weather"):
        return get_weather(user_input), None

    if "weather" in user_input.lower():
        return responses.get("weather"), {"awaiting_weather": True}


    if re.search(r'\d', user_input):
        try:
            result = eval(user_input)
            return f"The result of the calculation is: {result}", None
        except Exception as e:
            return "Sorry, I couldn't perform that calculation.", None

    # Handle various predefined responses
    for token in doc:
        if token.lemma_ == "hi" or token.lemma_ == "hello":
            return responses.get("greet"), None
        elif token.lemma_ == "bye":
            return responses.get("farewell"), None
        elif token.lemma_ == "help":
            return responses.get("help"), None
        elif token.lemma_ == "python":
            return responses.get("learn python"), None
        elif token.lemma_ == "joke":
            return responses.get("tell me a joke"), None
        elif token.lemma_ == "interesting":
            return responses.get("tell me something interesting"), None
        elif token.lemma_ == "name":
            return responses.get("name"), None
        elif token.lemma_ == "calculate":
            return responses.get("calculate"), None
        elif token.lemma_ == "ai":
            return responses.get("learn ai"), None
        elif "open" in user_input and "docs" in user_input:
            webbrowser.open("https://docs.python.org/3/")
            return responses.get("open python docs"), None


    return "Sorry, I don't understand that. Can you try asking something else?", None

# Function  weather information
def get_weather(city):
    try:
        url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
        response = requests.get(url)
        weather_data = response.json()

        if weather_data['cod'] == 200:
            city_name = weather_data['name']
            temp = weather_data['main']['temp']
            weather_desc = weather_data['weather'][0]['description']
            return f"The weather in {city_name} is currently {temp}°C with {weather_desc}."
        else:
            return "Sorry, I couldn't find the weather for that location."
    except Exception as e:
        return "Sorry, there was an error fetching the weather."

# Main chatbot loop
def chat():
    context = {}
    print("Chatbot: Hello! Type 'bye' to end the conversation.")

    while True:
        user_input = input("You: ")

        if user_input.lower() == 'bye':
            print("Chatbot: Goodbye!")
            break


        response, context_update = get_response(user_input, context)


        if context_update:
            context.update(context_update)
        else:
            context.clear()

        print(f"Chatbot: {response}")

chat() #call main function


Chatbot: Hello! Type 'bye' to end the conversation.
You: hi
Chatbot: Hello! How can I assist you today?
You: joke
Chatbot: Why do programmers prefer dark mode? Because light attracts bugs!
You: help
Chatbot: I'm here to help! Ask me anything.
You: interesting
Chatbot: Did you know that Python was named after Monty Python, not the snake?
You: python
Chatbot: You can start with this beginner's guide to Python: https://docs.python.org/3/tutorial/
You: ai
Chatbot: Check out this link to learn more about AI: https://www.coursera.org/courses?query=artificial%20intelligence
You: weather
Chatbot: Let me check the weather for you. Please provide the city name.
You: chennai
Chatbot: The weather in Chennai is currently 31.17°C with haze.
You: weather
Chatbot: Let me check the weather for you. Please provide the city name.
You: delhi
Chatbot: The weather in Delhi is currently 32.05°C with haze.
You: weather
Chatbot: Let me check the weather for you. Please provide the city name.
You: tenkasi
Chatb